In [1]:
from csv import reader
from collections import defaultdict 
import random

import sys
import os
import time

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")
    sys.path.append(module_path+"\\models")

import vectorizer
from decision_tree import BinaryDecisionTree
import evaluation

## Data processing functions

In [2]:
# identifier for the last used feature set, used for convenience, added to the file name of the results
used_feature_set = ""

# all of these functions take in the raw text rows and vocabulary 
# and outputs all instances formatted as feature vectors x and label vectors y
def features1(rows, word_to_index):
    """
    Features: is the word present in any of the two observations or hypothesis
    """
    global used_feature_set
    used_feature_set = "features1"
    # make training instances with the first hypothesis
    instances  = [(row[1] + " " + row[2] + " " + row[3], 1 if row[5] == '1' else 0) for row in rows]

    # add training instances with the second hypothesis
    instances += [(row[1] + " " + row[2] + " " + row[4], 1 if row[5] == '2' else 0) for row in rows]

    # change the text into sparse incidence vectors
    vectorized_instances = [(vectorizer.sparse_incidence_vector(text, word_to_index), label) for (text, label) in instances]

    # convert from list of (vector, label) tuples into two separate lists
    [x, y] = [list(t) for t in zip(*vectorized_instances)]

    return x, y, len(word_to_index)

def features2(rows, word_to_index):
    """
    Features: is the word present in both the hypothesis and either of the observations
    """

    global used_feature_set 
    used_feature_set = "features2"

    # make training instances with the first hypothesis
    instances  = [(row[1] + " " + row[2], row[3], 1 if row[5] == '1' else 0) for row in rows]

    # add training instances with the second hypothesis
    instances += [(row[1] + " " + row[2], row[4], 1 if row[5] == '2' else 0) for row in rows]

    # change the text into sparse incidence vectors
    vectorized_instances = [(vectorizer.sparse_incidence_vector(observations, word_to_index).intersection(vectorizer.sparse_incidence_vector(hypothesis, word_to_index)), label) for (observations, hypothesis, label) in instances]

    # convert from list of (vector, label) tuples into two separate lists
    [x, y] = [list(t) for t in zip(*vectorized_instances)]

    return x, y, len(word_to_index)

def features3(rows, word_to_index):
    """
    Features: two separate features for each word - is it present in either of the observations, is it present in the hypothesis
    """
    global used_feature_set 
    used_feature_set = "features3"
    
    # make training instances with the first hypothesis
    instances  = [(row[1] + " " + row[2], row[3], 1 if row[5] == '1' else 0) for row in rows]

    # add training instances with the second hypothesis
    instances += [(row[1] + " " + row[2], row[4], 1 if row[5] == '2' else 0) for row in rows]

    # change the text into sparse incidence vectors
    vectorized_instances = []
    for (observations, hypothesis, label) in instances:
        obs = vectorizer.sparse_incidence_vector(observations, word_to_index)
        hyp = vectorizer.sparse_incidence_vector(hypothesis, word_to_index)
        vectorized_instances.append((obs | {f + len(word_to_index) for f in hyp}, label))

    # convert from list of (vector, label) tuples into two separate lists
    [x, y] = [list(t) for t in zip(*vectorized_instances)]

    return x, y, len(word_to_index) * 2

In [3]:
# Functions to remove useless features. For the sake of keeping data processing steps separate, works with already calculated feature sets
def choose_features_to_remove(x, y, total_feature_amount, threshold = 1):
    """
    Counts up how often each feature occurs and returns a set of all the feature indexes that occur below the threshold
    """
    feature_counts = [0] * total_feature_amount
    for instance in x:
        for f in instance:
            feature_counts[f] += 1
    
    return {i for i, count in enumerate(feature_counts) if count < threshold}

def prune_features(x, y, features_to_remove):
    """
    Removes the features given in the set features_to_remove from x.
    Note that since features are stored densely with just their index, there's no point in trying to reindex the other features,
    since that wouldn't free up any space.
    """
    x = [features - features_to_remove for features in x]

    return x, y


def resample_dataset(train_rows, dev_rows, test_story_amount=1000):
    """
    Resamples the training and dev sets based on their story_id fields, so that each story id appears only once among both sets.
    Then samples a given amount of stories as the new dev set.

    Duplicate stories from the training data (that had the same observations, but different hypothesis) are removed, to make sure that 
    the training set has none of the stories that are in the dev set.
    """

    # get all unique story ids from both given data sets
    stories = {instance[0] for instance in train_rows}
    test_stories = {instance[0] for instance in dev_rows}
    all_stories = stories | test_stories

    # randomly sample new train/dev split among the story ids
    test_stories = random.sample(all_stories, k=test_story_amount)
    test_stories = {story_id for story_id in test_stories}
    train_stories = all_stories - test_stories

    new_dev_rows = []
    new_train_rows = []

    # divide all the data rows between dev and train sets, making sure to only take one row for each unique story_id
    for row in train_rows + dev_rows:        
        if row[0] in test_stories:
            new_dev_rows.append(row)
            test_stories -= {row[0]}
        elif row[0] in train_stories:
            new_train_rows.append(row)
            train_stories -= {row[0]}

    return new_train_rows, new_dev_rows


## Data loading and processing

In [4]:
feature_removal_threshold = 10

rows = vectorizer.parse_and_return_rows('../utils/data/processed_data/train.csv')
dev_rows = vectorizer.parse_and_return_rows('../utils/data/processed_data/dev.csv')

rows, dev_rows = resample_dataset(rows, dev_rows)

print(len({instance[0] for instance in rows}))
print(len(rows))

print(len({instance[0] for instance in dev_rows}))
print(len(dev_rows))


vocabulary, vocabulary_length = vectorizer.return_len_and_vocabulary(rows)
word_to_index = vectorizer.create_token_index(vocabulary)
x, y, total_feature_amount = features3(rows, word_to_index)
print(len(x), " instances, ", total_feature_amount, " features")
features_to_remove = choose_features_to_remove(x, y, total_feature_amount, feature_removal_threshold)
print("Pruning ", len(features_to_remove), " features")
x, y = prune_features(x, y, features_to_remove)
print(len(x), " instances with ", total_feature_amount-len(features_to_remove), " features remaining")


x_dev, y_dev, _ = features3(dev_rows, word_to_index)
x_dev, y_dev = prune_features(x_dev, y_dev, features_to_remove)
print(len(x_dev), " test instances")

<ipython-input-3-51d714f5f41a>:39: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_stories = random.sample(all_stories, k=test_story_amount)
18333
18333
1000
1000
36666  instances,  27712  features
Pruning  22208  features
36666  instances with  5504  features remaining
2000  test instances


## Training functions

In [5]:
def train_tree(max_depth, subset_size, training_instance_threshold, num_threads=1, print_logs=True, decision_tree=None, logs=[], accuracy_frequency=10):
    """
    Performs training of the decision tree one step at a time, while recording various statistics about the process.
    """
    start = time.time()

    # the decision tree instance can be passed as a parameter. 
    # This is so that training can be stopped manually without losing the trained decision tree instance.
    if decision_tree is None:
        decision_tree = BinaryDecisionTree()
    decision_tree.initialize_training(x, y)

    can_keep_expanding = True

    while can_keep_expanding:
        if max_depth is not None and decision_tree.current_depth >= max_depth:
            return logs, decision_tree # max depth reached, stop training early
        
        one_layer_start = time.time()

        # expand the next depth layer of the tree
        can_keep_expanding = decision_tree.expand_tree(subset_size, training_instance_threshold, num_threads)

        one_layer_end = time.time()
        
        layer_time = one_layer_end - one_layer_start
        total_time = one_layer_end - start

        if print_logs:
            print("Depth: ", decision_tree.current_depth, "Total nodes: ", decision_tree.total_nodes, "Time taken on layer: ", layer_time, "Total time taken: ", total_time)
        logs.append((decision_tree.current_depth, decision_tree.total_nodes, layer_time, total_time))

        # Calculating accuracy (especially on the training set) is a bit expensive, so we're not doing it every step.
        # This is only meant to check up on the progress, since the final accuracy will be calcuated at all depths simultaneously after training is done.
        if decision_tree.current_depth % accuracy_frequency == 0:
            print("dev: ", calculate_accuracy(decision_tree, x_dev, y_dev))
            print("train: ", calculate_accuracy(decision_tree, x, y))

    return logs, decision_tree

def calculate_accuracy(decision_tree, x, y):
    """
    Make a prediction on each instance in x with the given decision_tree, then calculate accuracy by comparing predictions to the labels in y.
    """
    predictions = []
    for instance in x:
        predictions.append(decision_tree.predict_class(instance))
    return evaluation.calculate_accuracy(predictions, y)

def calculate_accuracy_at_all_depths(decision_tree, x, y):
    """
    Make predictions on all instances in x with the given decision_tree at each depth.
    Returns a list of accuracies, where accuracies[i] is the accuracy that would be obtained if traversing the tree was stopped when it reached a depth of i.
    """
    accuracies = []

    # make predictions on each instance of x. We get a list of lists of size len(x) * depth
    predictions_at_all_depths = decision_tree.predict_classes_at_all_depths(x)

    # calculate transpose of predictions, so that it becomes depth * len(x)
    predictions_at_all_depths = list(map(list, zip(*predictions_at_all_depths)))

    # now when iterating through the predictions each row is a list of predicted classes for all instances in x
    for predictions_at_depth in predictions_at_all_depths:
        accuracy = evaluation.calculate_accuracy(predictions_at_depth, y)
        accuracies.append(accuracy)

    return accuracies

def save_results(logs, train_accuracies, dev_accuracies, file_name):
    """
    Saves training statistics in a file with the given name.
    """
    file = open(file_name, "w")
    file.write("acc_train,acc_dev,depth,nodes,time_for_layer,total_time\n")
    for i, log in enumerate(logs):
        message = "%s,%s,%s,%s,%s,%s\n" % (train_accuracies[i], dev_accuracies[i], log[0], log[1], log[2], log[3])
        file.write(message)
    file.close()

def do_experiment(max_depth, subset_size, training_instance_threshold, result_file_name, num_threads=1, print_logs=True, accuracy_frequency=10, tree=None, logs=[]):
    """
    Combines all the previous functions in a self contained test. Trains decision tree, calculates accuracies and saves the results.

    max_depth: when the tree reaches this depth, training will be stopped
    subset_size: how many training instances to look at when calculating information depth. Using only a subset speeds up training at the cost of accuracy.
    training_instance_threshold: normalizing measure. Don't split leaf nodes that have less than this amount of training instances associated with them.
    result_file_name: file name in which to store results
    num_threads: how many processes to use when calculating information gain. 5 works well on my computer, but will depend on hardware.
    accuracy_frequency: how often to calculate the accuracy and print it out. Only used for checking on the progress of the tree, full accuracy will be                                 calculated at the end anyway.
    tree, logs: in case the training is stopped before it's finished, if you passed your own tree/logs instances here, you'll still be able to access them 
                so the training progress won't be lost.
    """
    logs, tree = train_tree(max_depth, subset_size, training_instance_threshold, num_threads, print_logs, tree, logs, accuracy_frequency)
                            
    print("Calculating accuracy at all depths...")
    start = time.time()
    train_accuracies = calculate_accuracy_at_all_depths(tree, x, y)
    dev_accuracies = calculate_accuracy_at_all_depths(tree, x_dev, y_dev)
    end = time.time()
    print("Time taken: ", end - start)
    print("Final train accuracy: ", train_accuracies[-1])
    print("Final dev accuracy: ", dev_accuracies[-1])
    save_results(logs, train_accuracies, dev_accuracies, result_file_name)
    return logs, tree

In [42]:
tree = BinaryDecisionTree()
logs = []

identifier = "e10_resampled"
max_depth = 150
subset_size = None
training_instance_threshold = 100

file_name = "../experiment_results/decision_tree/%s_%s_prune%s_subset%s_threshold%s.csv" % (identifier, used_feature_set, feature_removal_threshold, subset_size, training_instance_threshold)

logs, tree = do_experiment(max_depth, subset_size, training_instance_threshold, result_file_name=file_name, num_threads=5,  accuracy_frequency=10, tree=tree, logs=logs)

Depth:  1 Total nodes:  3 Time taken on layer:  22.98327398300171 Total time taken:  22.98627543449402
Depth:  2 Total nodes:  7 Time taken on layer:  22.632786512374878 Total time taken:  45.6190619468689
Depth:  3 Total nodes:  13 Time taken on layer:  25.264089822769165 Total time taken:  70.88315176963806
Depth:  4 Total nodes:  19 Time taken on layer:  29.01190686225891 Total time taken:  99.89605855941772
Depth:  5 Total nodes:  29 Time taken on layer:  30.558051824569702 Total time taken:  130.45411038398743
Depth:  6 Total nodes:  41 Time taken on layer:  28.565892457962036 Total time taken:  159.02200078964233
Depth:  7 Total nodes:  57 Time taken on layer:  28.898897647857666 Total time taken:  187.9208984375
Depth:  8 Total nodes:  77 Time taken on layer:  28.84203290939331 Total time taken:  216.76393055915833
Depth:  9 Total nodes:  93 Time taken on layer:  28.842495918273926 Total time taken:  245.60742735862732
Depth:  10 Total nodes:  109 Time taken on layer:  28.761214

KeyboardInterrupt: 

In [ ]:
# use this to save the results of a run that was cancelled early  
print("Calculating accuracy at all depths...")
start = time.time()
train_accuracies = calculate_accuracy_at_all_depths(tree, x, y)
dev_accuracies = calculate_accuracy_at_all_depths(tree, x_dev, y_dev)
end = time.time()
print("Time taken: ", end - start)
print("Final train accuracy: ", train_accuracies[-1])
print("Final dev accuracy: ", dev_accuracies[-1])
save_results(logs, train_accuracies, dev_accuracies, file_name)
